In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from alphabase.spectral_library.base import SpecLibBase
from alphadia.extraction import data, planning
from alphadia.extraction.workflow import manager, peptidecentric

In [ ]:
performance_test_folder = '/Users/georgwallmann/Documents/data/performance_tests'

MODE = 'astral'

if MODE == 'astral':
    raw_files = [os.path.join(performance_test_folder, 'raw_data/astral_lf_dia/20230815_OA1_SoSt_SA_Whisper40_ADIAMA_HeLa_5ng_8Th14ms_FAIMS-40_1900V_noLoopCount_01.raw')]
    output_location = os.path.join(performance_test_folder, 'outputs/astral_lf_dia')
    speclib = os.path.join(performance_test_folder, 'libraries/astral/48_fraction_hela_PaSk_orbitrap_ms2.hdf')
elif MODE == 'timstof':
    raw_files = [os.path.join(performance_test_folder, 'raw_data/timstof_lf_diap/20230502_TIMS05_PaSk_SA_HeLa_21min_diaP_12scans_S2-A3_1_2089.d')]
    output_location = os.path.join(performance_test_folder, 'outputs/timstof_lf_diaPASEF')
    speclib = os.path.join(performance_test_folder, 'libraries/timstof/21min_Evosep_HeLa_BR14_48fractions_diaPASEF_py_diAID_2_egs.hdf')
else:
    raise ValueError('Unknown mode')

In [ ]:
test_lib = SpecLibBase()
test_lib.load_hdf(speclib, load_mod_seq=True)
plan = planning.Plan(output_location, raw_files, test_lib)

plan.config['general']['reuse_calibration'] = False
plan.config['general']['thread_count'] = 10
plan.config['general']['astral_ms1'] = False
plan.config['calibration']['norm_rt_mode'] = 'linear'

plan.config['extraction_target']['target_num_candidates'] = 5
plan.config['extraction_target']['target_ms1_tolerance'] = 3 if MODE == 'astral' else 15
plan.config['extraction_target']['target_ms2_tolerance'] = 5 if MODE == 'astral' else 15
plan.config['extraction_target']['target_rt_tolerance'] = 150

In [ ]:
for raw_name, dia_path, speclib in plan.get_run_data():
    pass

In [ ]:
workflow = peptidecentric.PeptideCentricWorkflow(
    raw_name,
    plan.config,
    dia_path,
    speclib
)
workflow.calibration()

In [ ]:
test_df = workflow.spectral_library.precursor_df.sample(1000)

In [ ]:
from alphadia.extraction import hybridselection

config = hybridselection.HybridCandidateConfig()
config.update(workflow.config['selection_config'])
config.update({
    'rt_tolerance':workflow.com.rt_error,
    'mobility_tolerance': workflow.com.mobility_error,
    'candidate_count': workflow.com.num_candidates,
    'precursor_mz_tolerance': workflow.com.ms1_error,
    'fragment_mz_tolerance': workflow.com.ms2_error,
    'exclude_shared_ions': workflow.config['library_loading']['exclude_shared_ions']
})

extraction = hybridselection.HybridCandidateSelection(
    workflow.dia_data.jitclass(),
    test_df,
    workflow.spectral_library.fragment_df,
    config.jitclass(),
    rt_column = f'rt_{workflow.com.column_type}',
    mobility_column = f'mobility_{workflow.com.column_type}',
    precursor_mz_column = f'mz_{workflow.com.column_type}',
    fragment_mz_column = f'mz_{workflow.com.column_type}',
    fwhm_rt = workflow.optimization_manager.fwhm_rt,
    fwhm_mobility = workflow.optimization_manager.fwhm_mobility
)
candidates_df = extraction(
    thread_count=workflow.config['general']['thread_count']
)

In [ ]:
from alphadia.extraction import plexscoring

config = plexscoring.CandidateConfig()
config.update(workflow.config['scoring_config'])
config.update({
    'precursor_mz_tolerance': workflow.com.ms1_error,
    'fragment_mz_tolerance': workflow.com.ms2_error,
    'exclude_shared_ions': workflow.config['library_loading']['exclude_shared_ions']
})

candidate_scoring = plexscoring.CandidateScoring(
    workflow.dia_data.jitclass(),
    workflow.spectral_library._precursor_df,
    workflow.spectral_library._fragment_df,
    config = config,
    rt_column = f'rt_{workflow.com.column_type}',
    mobility_column = f'mobility_{workflow.com.column_type}',
    precursor_mz_column = f'mz_{workflow.com.column_type}',
    fragment_mz_column = f'mz_{workflow.com.column_type}',
)

features_df, fragments_df = candidate_scoring(
    candidates_df, 
    thread_count=workflow.config['general']['thread_count'],
)